## loading data/libs

In [ ]:
import pandas as pd
import calendar
from bokeh.charts import output_notebook, Scatter, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.plotting import figure

output_notebook() 

In [ ]:
INPUT="data/device_failure.csv" 
dataset = pd.read_csv(INPUT,index_col=[0,1],parse_dates=[0])

In [ ]:
label_dset = dataset[["failure"]]


### checking devices

In [ ]:
total_failures_per_device = label_dset.groupby(level=1).agg(sum)
total_failures_per_device["failure"].value_counts()

Each device fail at least once

~10% device failing

'only' 106 positive points

### checking Dates

In [ ]:
dates = label_dset.index.get_level_values(0)
print "Range: from %s to %s" % (dates.min(), dates.max())


total_failures_per_date = label_dset.groupby(level=0).agg(sum)
print
print " n failures per date"
print str(total_failures_per_date["failure"].value_counts())
print
print "total: %i failures for %i days" % (total_failures_per_date["failure"].sum(), 
                                          total_failures_per_date[total_failures_per_date.failure >0].count())

In [ ]:
from bokeh.plotting import figure
data =total_failures_per_date.resample("M").sum() 
test = label_dset.reset_index("device").resample("M").count()
data["n_devices"] = test["device"]
data["failure_ratio_pm"] = data["failure"] / data["n_devices"] * 1000
data.index = (calendar.month_abbr[i] for i in data.index.month)
l = Line(
    data["failure_ratio_pm"],
    title="failures per Month",
    ylabel="n_failures", 
    xlabel="month"
)
show(l)

In [ ]:
weekday_dset = total_failures_per_date.copy()
weekday_dset.index = ["%i:%s" % (i,calendar.day_name[i]) for i in total_failures_per_date.index.weekday]

per_day = weekday_dset.groupby(level=0).sum()

print "failures per weekday"

per_day.sort_index()


- Long term trend with more failures in the past
- Less  failures over the weekend
- The absence of weekend could be explained by maintenance hapening only during workweek (hence explaing more failures on monday


### Per Device description 

In [ ]:
import numpy as np
dates = label_dset.swaplevel()
dd= dates.reset_index("date")["date"]
devices = pd.DataFrame({"min_date":dd.groupby(level=0).min()})
devices["max_date"] =  dd.groupby(level=0).max()
devices["n_lines"] = dd.groupby(level=0).count()
devices["n_days"] = (devices["max_date"] - devices["min_date"] ) /np.timedelta64(1, 'D') +1
devices["missing_values"] = devices["n_days"] - devices["n_lines"]

In [ ]:
devices["min_date"].value_counts()

### checking the nb devices per month. this is better done above

In [ ]:
#pd.DataFrame({"n_devices":devices["max_date"].dt.month.value_counts().sort_index()})
montlhy_devices = pd.DataFrame({"n_devices":devices["max_date"].dt.month.value_counts().sort_index()})
montlhy_devices.index = [calendar.month_abbr[i] for i in montlhy_devices.index]
montlhy_devices

### bucketing the n devices with missing day data

In [ ]:
i = ( (devices["missing_values"] //20)*20).value_counts()
#i = ( (devices["missing_values"])).value_counts(bins=10)
i.index.name = "n missing days"
pd.DataFrame({"n devices":i.sort_index()})

In [ ]:
i = devices["n_days"].value_counts(bins=10).sort_index()
i.index.name='n_days'
b = Bar(pd.DataFrame(
    {"n_devices":i}),
    xlabel="n days",
   title="devices distributed by ndays"
       )
show(b)

 ## features

In [ ]:
feature_dset = dataset[["attribute1"]]


In [ ]:
from bokeh.io import hplot
w = 450
p1 = figure(title="Attribute1 (μ=0, σ=0.5)",width=w)
hist, edges = np.histogram(dataset["attribute1"], density=True, bins=20)
hist0, edges0 = np.histogram(dataset[dataset["failure"]>0], density=True, bins=20)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")

p2 = figure(title="Attribute1 (μ=0, σ=0.5)",width=w)
p2.quad(top=hist0, bottom=0, left=edges0[:-1], right=edges0[1:],
        fill_color="lightblue", line_color="lightblue")
h = hplot(p1,p2)
show(h)

In [ ]:
show(p1)

In [ ]:
feature_dset["attribute1"].describe()

In [ ]:
dd = points["date"]
#.dt.month